### - 데이터 수집

#### # 최근 6개월 후기(제목, 시간, 별점, 날짜, 후기)

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
from datetime import datetime, timedelta
# Chrome WebDriver 경로 설정 (본인 환경에 맞게 수정 필요)
driver = webdriver.Chrome()
# 웹 페이지 로드
driver.get('https://edu.tsherpa.co.kr/LectureInfo/LectureReviews2')
# 데이터를 저장할 리스트 초기화
titles = []
times = []
ratings = []
dates = []
contents = []
# 오늘 날짜 기준으로 6개월 전 날짜 계산
today = datetime.now()
six_months_ago = today - timedelta(days=180)
page = 1
continue_crawling = True
while page <= 315 and continue_crawling:
    try:
        # 페이지 버튼의 XPath 계산
        if page % 10 == 1:
            page_xpath = '//*[@id="mainForm"]/div[3]/strong/a'
        else:
            page_in_group = (page - 1) % 10 + 2
            page_xpath = f'//*[@id="mainForm"]/div[3]/a[{page_in_group}]'
        # 페이지 버튼 클릭하여 해당 페이지로 이동
        page_button = WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.XPATH, page_xpath))
        )
        page_button.click()
        # 현재 페이지 번호 출력
        print(f"현재 페이지: {page}")
        # 각 페이지의 10개 요소 순회
        for row in range(1, 11):
            try:
                # 날짜 추출
                date_xpath = f'//*[@id="mainForm"]/table/tbody/tr[{row}]/td[3]'
                date_element = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, date_xpath))
                )
                date_text = date_element.text.strip().split('\n')[-1]
                review_date = datetime.strptime(date_text, '%Y.%m.%d')
                # 6개월 이내의 후기만 크롤링
                if review_date >= six_months_ago:
                    dates.append(date_text)
                    # 제목 추출
                    title_xpath = f'//*[@id="mainForm"]/table/tbody/tr[{row}]/td[2]/a'
                    title_element = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.XPATH, title_xpath))
                    )
                    title = title_element.text.split('<br>')[0].strip()
                    title = title.split('시간 ')[1].split('\n\n')[0]
                    titles.append(title)
                    # 본문 추출
                    content_xpath = f'//*[@id="mainForm"]/table/tbody/tr[{row}]/td[2]/a'
                    content_element = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.XPATH, content_xpath))
                    )
                    content_html = content_element.get_attribute('innerHTML')
                    content = content_html.split('<br>\n')[2].strip().split('<img')[0].strip()
                    contents.append(content)
                    # 시간 추출
                    time_xpath = f'//*[@id="mainForm"]/table/tbody/tr[{row}]/td[2]/a/span'
                    time_element = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.XPATH, time_xpath))
                    )
                    time = time_element.text.strip().replace('시간','')
                    times.append(time)
                    # 별점(평점) 추출
                    rating_xpath = f'//*[@id="mainForm"]/table/tbody/tr[{row}]/td[3]/img'
                    rating_element = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.XPATH, rating_xpath))
                    )
                    rating_alt = rating_element.get_attribute('alt')
                    rating_alt = rating_element.get_attribute('alt').replace('점', '')  # '점' 제거
                    ratings.append(rating_alt)
                else:
                    print(f"6개월 이내의 후기가 아닙니다: {date_text}")
                    continue_crawling = False
                    break
            except Exception as e:
                print(f"페이지 데이터 추출 중 오류 발생 (행 {row}):", e)
                continue
        # 다음 페이지 번호로 이동
        if page % 10 == 0 and page <= 310 and continue_crawling:  # 페이지 그룹의 마지막 페이지이고 310 페이지 이하인 경우
            next_page_button_xpath = '//*[@id="mainForm"]/div[3]/a[12]'
            next_page_button = WebDriverWait(driver, 30).until(
                EC.element_to_be_clickable((By.XPATH, next_page_button_xpath))
            )
            next_page_button.click()
        page += 1
    except Exception as e:
        print("전체 프로세스 중 오류 발생:", e)
        break
# 결과를 데이터프레임으로 변환
result_df = pd.DataFrame({
    '제목': titles,
    '시간': times,
    '별점': ratings,
    '날짜': dates,
    '본문': contents
})
# 결과 출력
print(result_df)
# 크롬 드라이버 종료
driver.quit()

현재 페이지: 1
현재 페이지: 2
현재 페이지: 3
현재 페이지: 4
현재 페이지: 5
현재 페이지: 6
현재 페이지: 7
현재 페이지: 8
현재 페이지: 9
현재 페이지: 10
현재 페이지: 11
현재 페이지: 12
현재 페이지: 13
현재 페이지: 14
현재 페이지: 15
6개월 이내의 후기가 아닙니다: 2023.12.19
                                    제목  시간 별점          날짜  \
0        과학저술가 정인경, 이지유와 함께하는 신나는 과학수다  15  5  2024.06.16   
1             쉽게 배우고 활용하는 교실 속 손그림 그리기  15  5  2024.06.15   
2    어린이 생활세계로 열어가는 사회 교과서 이야기 (5~6학년)  10  4  2024.06.15   
3                    교사를 위한 교실 밖 여행인문학  15  5  2024.06.15   
4                          초등 고학년 탐구생활  15  5  2024.06.15   
..                                 ...  .. ..         ...   
142           English Grammar in Use 2  30  5  2023.12.23   
143                  세상에서 제일 쉬운 과학 이야기  30  5  2023.12.22   
144  어린이 생활세계로 열어가는 사회 교과서 이야기 (5~6학년)  10  5  2023.12.22   
145           손그림과 파워포인트로 교실 속 콘텐츠 만들기  30  5  2023.12.21   
146                메이슨의 Travel English  30  5  2023.12.21   

                                      본문  
0     가벼운 대화로도 깊이 있게 생각할 거리를 주는 연수였습니다.

In [2]:
result_df

,제목,시간,별점,날짜,본문
0,"과학저술가 정인경, 이지유와 함께하는 신나는 과학수다",15,5,2024.06.16,가벼운 대화로도 깊이 있게 생각할 거리를 주는 연수였습니다.…
1,쉽게 배우고 활용하는 교실 속 손그림 그리기,15,5,2024.06.15,"손으로 끄적거리는 걸 좋아하는 편이지만,\n안타깝게도 예술적…"
2,어린이 생활세계로 열어가는 사회 교과서 이야기 (5~6학년),10,4,2024.06.15,오랫만에 고학년을 담임해서 \n사회과 지도가 만만치 않아서 …
3,교사를 위한 교실 밖 여행인문학,15,5,2024.06.15,여행을 잘 다니지 않는 방안퉁수형 인간인지라 여행지를 보면 …
4,초등 고학년 탐구생활,15,5,2024.06.15,10여년 만에 하는 고학년 두렵기만 했습니다. 지금은 어느 …
...,...,...,...,...,...
142,English Grammar in Use 2,30,5,2023.12.23,이지경선생님은 예전부터 문법으로 유명하다고 들었는데..\n티셀…
143,세상에서 제일 쉬운 과학 이야기,30,5,2023.12.22,어려운 과학 이야기를 이해하기 쉽게 풀어서 설명해주시어 재미…
144,어린이 생활세계로 열어가는 사회 교과서 이야기 (5~6학년),10,5,2023.12.22,짧은 시간의 연수지만 알차고 도움이 많이 되었습니다.감사합니…
145,손그림과 파워포인트로 교실 속 콘텐츠 만들기,30,5,2023.12.21,동료와 함께 그림을 그리며 함께 그리는 시간이되어 정말 재미…


#### # 직무연수 목록(제목, 연수 분야, 가격, 시간, 대상)

In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException
import pandas as pd

# 웹 드라이버 초기화
driver = webdriver.Chrome()

try:
    # 페이지 접속
    driver.get('https://edu.tsherpa.co.kr/Product/List/')
    
    # 데이터 저장을 위한 리스트 초기화
    titles = []
    times = []
    peoples = []
    pays = []
    fields = []
    
    # 각 요소에 대한 데이터를 추출
    for i in range(1, 82):  # 1부터 81까지
        try:
            field = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, f'//*[@id="mainForm"]/div[3]/div/ul/li[{i}]/div[1]'))
            ).text
            fields.append(field)
        except (TimeoutException, NoSuchElementException):
            fields.append(float('nan'))
        
        try:
            title = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, f'//*[@id="mainForm"]/div[3]/div/ul/li[{i}]/div[3]/p[2]/a'))
            ).text
            titles.append(title)
        except (TimeoutException, NoSuchElementException):
            titles.append(float('nan'))
        
        try:
            time = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, f'//*[@id="mainForm"]/div[3]/div/ul/li[{i}]/div[3]/p[1]/span[1]'))
            ).text.replace('시간', '')  # '시간' 제거
            times.append(time)
        except (TimeoutException, NoSuchElementException):
            times.append(float('nan'))
        
        try:
            price_elem = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, f'//*[@id="mainForm"]/div[3]/div/ul/li[{i}]/div[4]/p[1]'))
            )
            price_text = price_elem.text.strip()
            if "100%할인\n0 원" in price_text:
                price = "0"  # Set to "0" if 100% discount
            else:
                price = price_text.replace('원', '').replace(',', '').strip()  # '원' 및 ',' 제거
            pays.append(price)
        except (TimeoutException, NoSuchElementException):
            pays.append(float('nan'))
        
        try:
            target = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, f'//*[@id="mainForm"]/div[3]/div/ul/li[{i}]/div[3]/div/p[2]'))
            ).text
            peoples.append(target)
        except (TimeoutException, NoSuchElementException):
            peoples.append(float('nan'))

    # 데이터 프레임 생성
    df = pd.DataFrame({
        '연수 분야': fields,
        '제목': titles,
        '시간': times,
        '가격': pays,
        '대상': peoples
    })

    # 결과 출력
    print(df)

except WebDriverException as e:
    print("웹드라이버 오류:", e)

finally:
    driver.quit()


     연수 분야                             제목  시간     가격              대상
0     교과수업                 교사가 알고싶은 독립운동사  20      0              전체
1    교과서연수    실패 없는 과학 교과서 실험 (3~4학년 1학기)  10      0              초등
2   ICT정보화    쉽게 시작할 수 있는 노션과 ZOOM 활용 꿀팁!  30  70000              전체
3     인문교양           학생들과 함께 나누는 기후변화 이야기  30  70000              전체
4   ICT정보화  학급경영, 수업, 업무 - 노션으로 한 방에 해결하기  15  40000  초등 중등 교육전문직 일반
..     ...                            ...  ..    ...             ...
76    교과수업                 수업을 살리는 미술 레시피  30  70000        유치 초등 일반
77    교과수업                 수업을 살리는 꿀팁 레시피  30  70000        초등 교육전문직
78    자기계발                    나만의 여행책 만들기  30  70000              전체
79    교과수업                 수업을 살리는 놀이 레시피  30  70000     초등 교육전문직 일반
80    교과수업                교과수업에 활용하는 진로교육  30  70000     초등 중등 교육전문직

[81 rows x 5 columns]


In [22]:
pd.reset_option('display.max_rows')
df

,연수 분야,제목,시간,가격,대상
0,교과수업,교사가 알고싶은 독립운동사,20,0,전체
1,교과서연수,실패 없는 과학 교과서 실험 (3~4학년 1학기),10,0,초등
2,ICT정보화,쉽게 시작할 수 있는 노션과 ZOOM 활용 꿀팁!,30,70000,전체
3,인문교양,학생들과 함께 나누는 기후변화 이야기,30,70000,전체
4,ICT정보화,"학급경영, 수업, 업무 - 노션으로 한 방에 해결하기",15,40000,초등 중등 교육전문직 일반
...,...,...,...,...,...
76,교과수업,수업을 살리는 미술 레시피,30,70000,유치 초등 일반
77,교과수업,수업을 살리는 꿀팁 레시피,30,70000,초등 교육전문직
78,자기계발,나만의 여행책 만들기,30,70000,전체
79,교과수업,수업을 살리는 놀이 레시피,30,70000,초등 교육전문직 일반


#### # 최근 6개월동안의 후기 최종 데이터(제목, 연수 분야, 시간, 후기, 별점, 날짜, 가격, 대상)

In [6]:
fin_res = pd.merge(result_df, df, left_on='제목', right_on='제목')

In [7]:
fin_res = fin_res.drop(columns=['시간_y']).rename(columns={'시간_x':'시간'})

In [8]:
fin_res

,제목,시간,별점,날짜,본문,연수 분야,가격,대상
0,"과학저술가 정인경, 이지유와 함께하는 신나는 과학수다",15,5,2024.06.16,가벼운 대화로도 깊이 있게 생각할 거리를 주는 연수였습니다.…,자기계발,40000,전체
1,쉽게 배우고 활용하는 교실 속 손그림 그리기,15,5,2024.06.15,"손으로 끄적거리는 걸 좋아하는 편이지만,\n안타깝게도 예술적…",ICT정보화,40000,전체
2,쉽게 배우고 활용하는 교실 속 손그림 그리기,15,5,2024.05.09,늘 그림을 잘 그리고 싶었던 시절이 오래전부터 있었습니다. …,ICT정보화,40000,전체
3,쉽게 배우고 활용하는 교실 속 손그림 그리기,15,5,2024.04.25,"활동지를 만들거나, 게시판을 꾸밀 때 직접 그린 손그림으로 …",ICT정보화,40000,전체
4,쉽게 배우고 활용하는 교실 속 손그림 그리기,15,5,2024.04.23,제목과 같이 정말 쉽게 배우고 활용할 수 있는 손그림 수업이…,ICT정보화,40000,전체
...,...,...,...,...,...,...,...,...
136,"학급경영, 수업, 업무 - 노션으로 한 방에 해결하기",15,5,2023.12.26,목소리가 편안하고 설명이 명료해서 듣기 편했어요.\n두 선생…,ICT정보화,40000,초등 중등 교육전문직 일반
137,"학급경영, 수업, 업무 - 노션으로 한 방에 해결하기",15,4,2023.12.25,"너무나 많은 앱들이 있어 사용하기가 버거웠는데, 이제 노션으…",ICT정보화,40000,초등 중등 교육전문직 일반
138,"학급경영, 수업, 업무 - 노션으로 한 방에 해결하기",15,5,2023.12.25,업무에도 하나씩 활용해 보니 효율성이 좋아졌습니다.,ICT정보화,40000,초등 중등 교육전문직 일반
139,나만의 여행책 만들기,30,5,2024.01.19,세계 여행을 다녀온 것 같은 기분이 드는 연수입니다.\n사진…,자기계발,70000,전체


In [10]:
# 최근 6개월 후기 csv 파일로 저장
fin_res.to_csv('./T_review.csv', index=False, encoding='utf-8-sig')

---

### - 기초 분석

#### # 연수 과정별 수강후기 개수

In [11]:
title_counts = fin_res['제목'].value_counts().reset_index()
title_counts.columns = ['제목', '수강후기 개수']

In [23]:
title_counts

,제목,수강후기 개수
0,쉽게 시작할 수 있는 노션과 ZOOM 활용 꿀팁!,8
1,세상에서 제일 쉬운 과학 이야기,6
2,쉽게 배우고 활용하는 교실 속 손그림 그리기,6
3,교사가 알고싶은 독립운동사,6
4,"학급경영, 수업, 업무 - 노션으로 한 방에 해결하기",6
5,지리와 함께하는 인문학 티타임,6
6,손그림과 파워포인트로 교실 속 콘텐츠 만들기,5
7,메이슨의 Travel English,5
8,"기후 변화 이야기, 교실에 오다",5
9,"행복한 학급 만들기, 학생과 교사 자존감 수업",4


#### # 연수 분야별 수강후기 개수

In [25]:
field_counts = fin_res['연수 분야'].value_counts().reset_index()
field_counts.columns = ['연수 분야', '후기 개수']

field_counts

,연수 분야,후기 개수
0,자기계발,43
1,ICT정보화,27
2,교과수업,26
3,어학,19
4,인문교양,12
5,생활지도,7
6,교과서연수,5
7,학습지도,1
8,학급경영,1


### 별점 분석

In [26]:
import plotly.express as px

In [27]:
star = fin_res.groupby('별점').count()
star

,제목,시간,날짜,본문,연수 분야,가격,대상
별점,,,,,,,
1,3,3,3,3,3,3,3
3,6,6,6,6,6,6,6
4,13,13,13,13,13,13,13
5,119,119,119,119,119,119,119


In [28]:
px.pie(values=star['제목'], names=star.index,title='T셀파 후기 별점 점유율')

In [31]:
star = fin_res[fin_res['별점']=='1']
star

,제목,시간,별점,날짜,본문,연수 분야,가격,대상
30,손그림과 파워포인트로 교실 속 콘텐츠 만들기,30,1,2024.04.22,너무 어려워요,ICT정보화,70000,전체
63,지리와 함께하는 인문학 티타임,30,1,2024.04.12,연수 후기가 좋아서 신청했는데 첫 강의부터 너무나 티나게 진…,자기계발,70000,전체
110,"하루 20분, 컬쳐 리스닝 픽업",15,1,2024.04.23,좋은 후기들이 많지만 솔직히 이해되지 않네요.\n교안도 없고…,어학,40000,전체


### 강좌 시간대가 15차시인 강의의 가격대 분포

In [36]:
time_15 = fin_res[fin_res['시간'] == '15']
time_15

,제목,시간,별점,날짜,본문,연수 분야,가격,대상
0,"과학저술가 정인경, 이지유와 함께하는 신나는 과학수다",15,5,2024.06.16,가벼운 대화로도 깊이 있게 생각할 거리를 주는 연수였습니다.…,자기계발,40000,전체
1,쉽게 배우고 활용하는 교실 속 손그림 그리기,15,5,2024.06.15,"손으로 끄적거리는 걸 좋아하는 편이지만,\n안타깝게도 예술적…",ICT정보화,40000,전체
2,쉽게 배우고 활용하는 교실 속 손그림 그리기,15,5,2024.05.09,늘 그림을 잘 그리고 싶었던 시절이 오래전부터 있었습니다. …,ICT정보화,40000,전체
3,쉽게 배우고 활용하는 교실 속 손그림 그리기,15,5,2024.04.25,"활동지를 만들거나, 게시판을 꾸밀 때 직접 그린 손그림으로 …",ICT정보화,40000,전체
4,쉽게 배우고 활용하는 교실 속 손그림 그리기,15,5,2024.04.23,제목과 같이 정말 쉽게 배우고 활용할 수 있는 손그림 수업이…,ICT정보화,40000,전체
5,쉽게 배우고 활용하는 교실 속 손그림 그리기,15,5,2024.01.17,칠판에 판서를 할때 글보다 그림이 더 효과적이고 특히 사회 …,ICT정보화,40000,전체
6,쉽게 배우고 활용하는 교실 속 손그림 그리기,15,4,2024.01.13,쉽고 재미있게 손 그림을 그릴 수 있도록 도움이 되는 연수였…,ICT정보화,40000,전체
10,교사를 위한 교실 밖 여행인문학,15,5,2024.06.15,여행을 잘 다니지 않는 방안퉁수형 인간인지라 여행지를 보면 …,인문교양,40000,전체
11,교사를 위한 교실 밖 여행인문학,15,5,2024.06.10,저는 여행에 관심이 많은 교사입니다. 학기중 바쁜 일정으로 …,인문교양,40000,전체
12,교사를 위한 교실 밖 여행인문학,15,4,2023.12.26,여행은 답답한 일상에서 오는 스트레스를 해소해 줍니다. 그러…,인문교양,40000,전체


In [40]:
time_15_count = time_15.groupby('가격').count()
time_15_count

,제목,시간,별점,날짜,본문,연수 분야,대상
가격,,,,,,,
0,1,1,1,1,1,1,1
40000,51,51,51,51,51,51,51


In [41]:
px.pie(values=time_15_count['제목'], names=time_15_count.index, title='T셀파 15차시 가격 점유율')

### 강좌 시간대가 30차시인 강의의 가격대 분포

In [42]:
time_30 = fin_res[fin_res['시간'] == '30']
time_30

,제목,시간,별점,날짜,본문,연수 분야,가격,대상
15,수업을 살리는 체육 레시피,30,5,2024.06.13,아이들이 정~~~말 좋아하는 체육을 더 더 즐겁게 해주기 위…,교과수업,70000,유치 초등 교육전문직 일반
16,수업을 살리는 체육 레시피,30,5,2024.04.09,간혹 학교에 물품이 없어서 하지 못하는 체육 활동들에 대한 …,교과수업,70000,유치 초등 교육전문직 일반
17,수업을 살리는 체육 레시피,30,5,2024.01.22,새학기 재미있는 체육 수업을 위해 ~,교과수업,70000,유치 초등 교육전문직 일반
18,수업을 살리는 체육 레시피,30,5,2024.01.16,수업하다가 학생들이 가끔 싫어하는 단원들도 있어 이로 응용해…,교과수업,70000,유치 초등 교육전문직 일반
21,그림검사와 다양한 기법으로 만나는 미술치료,30,5,2024.06.07,학생들이 감정이나 내면세계를 미술 활동을 통해 표현하므로 학…,자기계발,70000,전체
...,...,...,...,...,...,...,...,...
127,7인의 전문의와 함께하는 메디컬 콘서트,30,4,2024.04.05,해마다 120시간 이상의 연수를 듣는 저로서는 올해 처음으로…,인문교양,70000,전체
128,7인의 전문의와 함께하는 메디컬 콘서트,30,3,2024.01.13,저는 좀 아쉬웠어요.\n좀 오래된 컨텐츠를 가져와 짜깁기인데…,인문교양,70000,전체
131,"나이와 신분을 초월한 독립운동, 그 현장을 읽다",30,5,2024.03.10,나라가 어려울 때 진정성 있는 마음으로 나라를 지켜줄 수 있…,자기계발,0,전체
132,"나이와 신분을 초월한 독립운동, 그 현장을 읽다",30,5,2024.01.25,독립 그 가슴뭉클한 단어에 대해 더 자세히 알 수 있었습니다…,자기계발,0,전체


In [43]:
time_30_count = time_30.groupby('가격').count()
time_30_count

,제목,시간,별점,날짜,본문,연수 분야,대상
가격,,,,,,,
0,2,2,2,2,2,2,2
70000,76,76,76,76,76,76,76


In [45]:
px.pie(values=time_30_count['제목'], names=time_30_count.index, title='T셀파 30차시 가격 점유율')